In [2]:
import pandas as pd
import numpy as np

df = pd.read_json('data/train.jsonl', lines=True)
df.head()

,ners,sentences,id
0,"[[0, 5, CITY], [16, 23, PERSON], [34, 41, PERS...",Бостон взорвали Тамерлан и Джохар Царнаевы из ...,0
1,"[[21, 28, PROFESSION], [53, 67, ORGANIZATION],...",Умер избитый до комы гитарист и сооснователь г...,1
2,"[[0, 4, PERSON], [37, 42, COUNTRY], [47, 76, O...",Путин подписал распоряжение о выходе России из...,2
3,"[[0, 11, PERSON], [36, 47, PROFESSION], [49, 6...",Бенедикт XVI носил кардиостимулятор\nПапа Римс...,3
4,"[[0, 4, PERSON], [17, 29, ORGANIZATION], [48, ...",Обама назначит в Верховный суд латиноамериканк...,4


In [2]:
def get_rid_of_overlaps(old_ners):
    old_ners.sort()
    new_ners = [tuple(old_ners[0])]
    i = 1
    while i != len(old_ners):
        if (old_ners[i][0] <= new_ners[-1][0] and new_ners[-1][1] <= old_ners[i][1]) or\
            (old_ners[i][0] <= new_ners[-1][0] <= old_ners[i][1]) or\
            (new_ners[-1][0] <= old_ners[i][0] <= new_ners[-1][1]) or\
            (old_ners[i][0] <= new_ners[-1][1] <= old_ners[i][1]) or\
            (new_ners[-1][0] <= old_ners[i][1] <= new_ners[-1][1]):
            new_ners[-1] = tuple(old_ners[i])
        elif not (new_ners[-1][0] <= old_ners[i][0] and old_ners[i][1] <= new_ners[-1][1]):
            new_ners.append(tuple(old_ners[i]))
            
        i += 1
        
    return new_ners

In [4]:
train_data = []
for i, row in df.iterrows():
    train_data.append((
        row['sentences'],
        {
            'entities': [(start, end + 1, ent) for start, end, ent in get_rid_of_overlaps(row['ners'])]
        } 
    ))

In [ ]:
import spacy

model = spacy.load('ru_core_news_sm')

In [ ]:
from tqdm.notebook import tqdm
import random
from spacy.training.example import Example

EPOCHS = 10
BATCH_SIZE = 64

for epoch in range(EPOCHS):
    random.shuffle(train_data)
    losses = {}
    for batch in tqdm(spacy.util.minibatch(train_data, size=BATCH_SIZE), total=len(train_data) // BATCH_SIZE + 1):
        examples = []
        for text, ann in batch:
            doc = model.make_doc(text)
            ex = Example.from_dict(doc, ann)
            examples.append(ex)
        model.update(examples, losses=losses)
    print(losses)

In [109]:
test_df = pd.read_json('data/dev.jsonl', lines=True)
test_df.head()

,senences,id
0,Генерал Д.Петреус назначен на пост главы ЦРУ.\...,519
1,Подозреваемые в нападении на Charlie Hebdo зах...,520
2,Скончалась Джанет Рено — первая женщина-генпро...,521
3,Школьник из Иванова получил «Золотой крест» за...,522
4,Врачи установили причину смерти Сергея Доренко...,523


In [180]:
test_df.iloc[1]['senences']

'Подозреваемые в нападении на Charlie Hebdo захватили заложника\nПанорама Даммартен-ан-Гоэль (Dammartin-en-Goële)\nПолицейское оцепление и журналисты\nВ городе Даммартен-ан-Гоэль (Dammartin-en-Goële; столица графства Даммартен) к северо-востоку от Парижа французские силовики проводят операцию по поимке двух подозреваемых в убийстве 12 человек в редакции журнала Charlie Hebdo.\n\nВ условиях дождливой погоды вертолеты, полицейские и пожарные машины окружили промышленный парк в городе, расположенном в 40 километрах от Парижа.\n\nПолиция подтвердила сообщения, что подозреваемые взяли по крайней мере одного заложника.\n\nВсе выезды из города закрыты. Мэр города призвал местных жителей не выходить на улицу. Вблизи Даммартен-ан-Гоэля расположен международный аэропорт имени Шарля де Голля.\n\nМинистр внутренних дел Франции Бернар Казнев, подтвердил, что власти проводят операцию по захвату подозреваемых. Казнев выступил с этим заявлением после срочной встречи с президентом, премьер-министром и 

In [181]:
test_text = df.iloc[0]['sentences']
test_answer = str(df.iloc[0]['ners'])

In [184]:
len(test_answer)

1628

In [125]:
subm = []

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    ners = {}
    doc = model(row['senences'])
    ners['ners'] = [[e.start_char, e.end_char - 1, e.label_] for e in doc.ents]
    ners['id'] = row['id']
    subm.append(ners)

  0%|          | 0/65 [00:00<?, ?it/s]

In [126]:
import json

with open("test.jsonl", 'w') as f:
    for item in subm:
        f.write(json.dumps(item) + "\n")